# Resolução de puzzles Sokoban

## Formulação

### Atributos

Foi escolhido representar um estado do puzzle sokoban usando uma class `EstadoSokoban` que tem como atributos:

- `tabuleiro`: lista de listas que representa o puzzle: Cada uma das listas interiores vão ter:

    - ’#’   – para representar as paredes;
    
    - ’.’   – para representar as posições livres;
    
    - ’*’   – para representar as caixas;
    
    - ’o’   – (um ó minúsculo) para representar os alvos;
    
    - ’A’   – para representar o arrumador.
    
    - ’@’   – para representar uma caixa numa posição alvo.
    
    - ’B’   – para representar o arrumador em cima de uma posição alvo.
    
- `arrumador`: posição do arrumador, tuplo xy.

- `caixas`: posição das caixas, tuplo xy.

- `alvos`: posição dos alvos, tuplo xy.

- `deadlocks`: posição de deadlocks, executando `deadlocks_tabuleiro()`

### Métodos

Para além da representação, foram implementados os seguintes métodos:

- `pos_livre(self, x, y)`: verifica se uma posição do puzzle está livre, isto é, se o arrumador por andar para lá.

- `pos_caixa(self, x, y)`: verifica se uma posição do puzzle tem uma caixa.

- `ver_*(self, x, y)`: usa as funções anteriores para verificar a posição "cima", "baixo", "direita" e "esquerda" (cada uma destas opções são um método diferente.

## Nota sobre este relatório

Utilizamos a ferramenta Jupyter Notebook, para tirar proveito de algumas funcionalidades do IPython para análise estatística da execução dos algoritmos.

## Heurísticas definidas

Considere-se $n$ caixas e $m$ alvos. No jogo sokoban, $n=m$ para qualquer jogo. 

Vamos então construir um grafo bipartido completo $K_{n,m}$.

Ao associar um custo às arestas de cada nodo caixa a um nodo alvo (neste caso baseado na distância de manhattan), passamos a ter um problema de afetação, para o qual existe uma série de algoritmos que, sabendo que cada nodo n só vai estar asssociado a um nodo m, permitem escolher qual a combinação de menor custo.

Nesta heurística, foi escolhido o algoritmo hungáro (dado que é um algoritmo estudado no âmbito das cadeira de Investigação Operacional e Grafos e Redes) para associar uma caixa a um alvo, baseado na melhor combinação de custos (Minimizar o custo global de $K_{n,m}$).

A complexidade temporal deste algoritmo é de $O(n^3)$, que é bastante pesado computacional- mente, mas visto que nos puzzles dados n ≤ 3, vai ser possivel calcular este algoritmo em tempo útil, como vamos testar na proximá secção.
Para não implementar o algoritmo hungáro de raíz, foi utilizada a implementação [1]: https://github.com/bmc/munkres.
Todo o código necessário está disponivel no ficheiro hungarian.py

## Exemplos de execução

No ficheiro `sokoban.py` estão as classes principais para a execução do puzzle. Todo o código de análise execução (apresentado neste relatório) está no ficheiro `run-sokoban.py`.

In [1]:
%run -t sokoban.py


IPython CPU timings (estimated):
  User   :       0.01 s.
  System :       0.00 s.
Wall time:       0.01 s.


## Análise dos algoritmos experimentados

Vai ser utilizado o puzzle2, entregue no enunciado, para testar os algorimos disponíveis no ficheiro `search.py`, do repositório aima-python, disponibilizado nas aulas.

A função (definida em baixo) `statistics` é método  para imprimir dados da resolução de um problema Sokoban.

In [11]:
sokoban = Sokoban(puzzle2)

def statistics(resultado, caminho=False):
    path = resultado.path()
    solucao = resultado.solution()
    number_moves = 0
    number_pushes = 0

    for index, action in enumerate(solucao):
        accao, _ = action.split()
        if accao == 'andar':
            number_moves += 1
        else:
            number_pushes += 1

    for index, state in enumerate(path):
        if caminho:
            print(state)
    else:
        print('Número de passos:', index)

    print('Números de moves:', number_moves)
    print('Números de pushes:', number_pushes)

In [3]:
%%timeit 
ucs_resultado = uniform_cost_search(sokoban)

1.73 s ± 223 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%%timeit
bfs_resultado = breadth_first_search(sokoban)

1.7 s ± 254 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
bfs_resultado = breadth_first_search(sokoban)
statistics(bfs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [6]:
ucs_resultado = uniform_cost_search(sokoban)
statistics(ucs_resultado)

Número de passos: 43
Números de moves: 31
Números de pushes: 12


In [7]:
%%timeit
astar_resultado = astar_search(sokoban, hung_alg_manh)

1.48 s ± 49.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
from hungarian import Munkres

def hung_alg_manh(nodo):
    """Algoritmo hungaro, em que o custo de cada caixa a um alvo é a distância de manhattan."""
    m = Munkres()
        
    caixas = nodo.state.caixas
    alvos = nodo.state.alvos
    custo = list()
    mhd = 0

    for index, c in enumerate(caixas):
        custo.append(list())
        for a in alvos:
            custo[index].append(abs(c[0] - a[0]) + abs(c[1] - a[1]))
    
    indexes = m.compute(custo)
    for row, column in indexes:
        value = custo[row][column]
        mhd += value
    
    return mhd

In [9]:
astar_resultado = astar_search(sokoban, hung_alg_manh)
statistics(astar_resultado, caminho=True)

#######
##    #
##* o #
#  A  #
# *#o #
#  ####
#######

#######
##    #
##* o #
# A   #
# *#o #
#  ####
#######

#######
##    #
##* o #
#A    #
# *#o #
#  ####
#######

#######
##    #
##* o #
#     #
#A*#o #
#  ####
#######

#######
##    #
##* o #
#     #
# *#o #
#A ####
#######

#######
##    #
##* o #
#     #
# *#o #
# A####
#######

#######
##    #
##* o #
# *   #
# A#o #
#  ####
#######

#######
##    #
##* o #
# *   #
#A #o #
#  ####
#######

#######
##    #
##* o #
#A*   #
#  #o #
#  ####
#######

#######
##    #
##* o #
# A*  #
#  #o #
#  ####
#######

#######
##    #
##* o #
#  A* #
#  #o #
#  ####
#######

#######
##    #
##*Ao #
#   * #
#  #o #
#  ####
#######

#######
##    #
##* B #
#   * #
#  #o #
#  ####
#######

#######
##    #
##* oA#
#   * #
#  #o #
#  ####
#######

#######
##    #
##* o #
#   *A#
#  #o #
#  ####
#######

#######
##    #
##* o #
#   * #
#  #oA#
#  ####
#######

#######
##    #
##* o #
#   * #
#  #B #
#  ####
#######

#######
##    #
##* @ #
#   A #